In [1]:
%load_ext autoreload
%autoreload 2

<h1>Оглавление<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Data-loader" data-toc-modified-id="Data-loader-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Data loader</a></span></li></ul></div>

In [2]:
from data.data_preparer import get_datasets

In [3]:
torch_dataset, torch_dataset_test = get_datasets()

Found annotations at '/mnt/heap/raw/instances_val2017.json'
Images already downloaded
Existing download of split 'validation' is sufficient
Loading 'coco-2017' split 'validation'
 100% |███████████████| 5000/5000 [22.6s elapsed, 0s remaining, 246.6 samples/s]      
Dataset 'coco-2017-validation' created
person_view len = 1005
train dataset = 753
test dataset = 252


In [4]:
import seaborn as sns
import pandas as pd

# Train loop

In [5]:
from itertools import product
from pytorch_helper import PyTorchHelper

In [6]:
from extended_model import ExtendedModel

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision.models import resnet18

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.model = resnet18(pretrained=True)
        self.model.fc = nn.Linear(in_features=512, out_features=5)
        
    def forward(self, x):
        x = self.model(x)
        return x

In [9]:
config = {
    'need_train': True,
    'regs': [0.001],
    'optimizers': [optim.Adam],
    'model': Net,
    'model_name': 'best_lenet'    
}

In [10]:
from resource_monitor import ResourceMonitor
resourceMonitor = ResourceMonitor()

In [11]:
resourceMonitor.print_statistics('MB')

memory_allocated: 0.0
max_memory_allocated: 0.0
memory_reserved: 0.0
max_memory_reserved: 0.0


In [12]:
import ray
from ray import tune

In [13]:
==========================================

SyntaxError: invalid syntax (<ipython-input-13-ae5601bd6a65>, line 1)

In [14]:
from runer import find_hyperparameters

In [ ]:
analysis = tune.run(
    tune.with_parameters(find_hyperparameters,  data_train=torch_dataset, data_test=torch_dataset_test),
    sync_config=tune.SyncConfig(
         syncer=None  # Disable syncing
    ),
    name="experiment_name",
    local_dir="/mnt/heap/My folder/checpoint",
    num_samples=1,
    config=config,
    resources_per_trial={"cpu": 8, "gpu": 1})

Initializing Ray automatically.For cluster usage or custom Ray initialization, call `ray.init(...)` before `tune.run`.


In [ ]:
extended_model = find_hyperparameters(config)

In [ ]:
from matplotlib import pyplot as plt

In [ ]:
viewer.visualize_loss_history(extended_model)

In [ ]:
viewer.visualize_metric_history(extended_model)

In [ ]:
resourceMonitor.print_statistics('MB')

In [ ]:
===============================================

In [ ]:
extended_model.save_best_model()

In [ ]:
extended_model.deployment_best_model()

In [ ]:
===============================================

In [ ]:
def predict(index):
    img, target = torch_dataset[index]
    pred = extended_model.torch_model(img.unsqueeze(0))[0]
    
    original_img = Image.open(target['img_path']).convert("RGB")
    viewer.print_prediction(original_img, target, pred)

In [ ]:
for i in range(10, 170):
    predict(i)

In [ ]:
===============================================

In [ ]:
===============================================

# Visualization

In [ ]:
import fiftyone as fo

def convert_torch_predictions(preds, det_id, s_id, w, h, classes):
    # Convert the outputs of the torch model into a FiftyOne Detections object
    dets = []
    #if preds[0] < 0.5:
     #   detections = fo.Detections(detections=dets)
      #  return detections, det_id
    
    # Parse prediction into FiftyOne Detection object
    x0,y0,x1,y1 = preds[1:]
    coco_obj = fouc.COCOObject(det_id, s_id, int(1), [x0, y0, x1-x0, y1-y0])
    det = coco_obj.to_detection((w,h), classes)
    dets.append(det)
    det_id += 1
        
    detections = fo.Detections(detections=dets)
        
    return detections, det_id

def add_detections(model, torch_dataset, view, field_name="predictions"):
    # Run inference on a dataset and add results to FiftyOne
    torch.set_num_threads(1)
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print("Using device %s" % device)

    model.eval()
    model.to(device)
    image_paths = torch_dataset.img_paths
    classes = torch_dataset.classes
    det_id = 0
    
    with fo.ProgressBar() as pb:
        for img, targets in pb(torch_dataset):
            sample = view[targets["img_path"]]
            s_id = sample.id
            w = sample.metadata["width"]
            h = sample.metadata["height"]
            
            # Inference
            preds = model(img.unsqueeze(0).to(device))[0]
            
            detections, det_id = convert_torch_predictions(
                preds, 
                det_id, 
                s_id, 
                w, 
                h, 
                classes,
            )
            
            sample["predictions"] = detections
            sample.save()

In [ ]:
add_detections(model, torch_dataset_test, fo_dataset)

In [ ]:
session = fo.launch_app(fo_dataset)